In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.optimizers import SGD

In [2]:
true = pd.read_csv('True.csv')
fake = pd.read_csv('Fake.csv')
true['label'] = 1
fake['label'] = 0
frames = [true.loc[:1000][:], fake.loc[:1000][:]]  #Edit this to control the size of dataset
df = pd.concat(frames)
df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [3]:
X = df.drop('label', axis=1) 
y = df['label']
df = df.dropna()
df2 = df.copy()
df2 = df2.reset_index(drop = True)


In [4]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(df2)):
    review = re.sub('[^a-zA-Z]', ' ', df2['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v = TfidfVectorizer(max_features=5000, ngram_range=(1,3))
X = tfidf_v.fit_transform(corpus).toarray()
y = df2['label']

In [6]:
def baseline_model() : 
    model = Sequential()
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dropout(0.5))
    model.add(Dense(80, activation = 'relu', kernel_initializer = 'normal'))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'normal'))
    
    sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

    

In [7]:
model = baseline_model()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model.fit(X_train, y_train, epochs=25, batch_size=64)
results = model.evaluate(X_test, y_test)

print(results)


Epoch 1/25
26/26 [==============================] - 4s 16ms/step - loss: 0.6528 - accuracy: 0.6052
Epoch 2/25
26/26 [==============================] - 0s 18ms/step - loss: 0.1576 - accuracy: 0.9781
Epoch 3/25
26/26 [==============================] - 0s 17ms/step - loss: 0.0030 - accuracy: 0.9988
Epoch 4/25
26/26 [==============================] - 0s 16ms/step - loss: 0.0043 - accuracy: 0.9988
Epoch 5/25
26/26 [==============================] - 0s 16ms/step - loss: 4.4182e-04 - accuracy: 1.0000
Epoch 6/25
26/26 [==============================] - 0s 17ms/step - loss: 2.9861e-04 - accuracy: 1.0000
Epoch 7/25
26/26 [==============================] - 0s 17ms/step - loss: 2.7413e-04 - accuracy: 1.0000
Epoch 8/25
26/26 [==============================] - 0s 16ms/step - loss: 1.7336e-04 - accuracy: 1.0000
Epoch 9/25
26/26 [==============================] - 0s 16ms/step - loss: 9.7854e-05 - accuracy: 1.0000
Epoch 10/25
26/26 [==============================] - 0s 16ms/step - loss: 1.5331e-04 - ac

In [10]:
import pickle 
from keras.models import load_model
# modelname = 'final_model.pkl'
vectorname = 'vectorizer.pkl'
pickle.dump(tfidf_v, open(vectorname, 'wb'))
model.save('nn_model.keras')
# pickle.dump(model, open(modelname, 'wb'))